In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, generate_data_commitment

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')
data_commitment_path = os.path.join('shared/data_commitment.json')
precal_witness_path = os.path.join('shared/precal_witness_arr.json')

=======================  ZK-STATS FLOW =======================

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')
create_dummy(data_path, dummy_data_path)

In [5]:
scales = [4]
selected_columns = ['col_name']
generate_data_commitment(data_path, scales, data_commitment_path)

In [6]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    # FIXME: should be replaced by `s.where` when it's available. Now the result may be incorrect
    filter = (x > 30)
    filtered_x = s.where(filter, x)
    return s.stdev(filtered_x)

error = 0.05
_, prover_model = computation_to_model(computation,precal_witness_path, True, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:254: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Desktop/zk-stats-lib/zkstats/ops.py:34: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return torch.tensor(True)
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is depre

==== Generate & Calibrate Setting ====




 <------------- Numerical Fidelity Report (input_scale: 4, param_scale: 4, scale_input_multiplier: 1) ------------->

+-------------+--------------+-------------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error  | median_error | max_error   | min_error | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+-------------+--------------+-------------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| 0.004298687 | 0.008597374  | 0.008597374 | 0         | 0.004298687    | 0.008597374      | 0.008597374   | 0             | 0.00003695742      | 0.00033692474      | 0.00033692474          |
+-------------+--------------+-------------+-----------+----------------+------------------+---------------+----

scale:  [4]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":4,"param_scale":4,"scale_rebase_multiplier":1,"lookup_range":[-140658,140920],"logrows":19,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Fixed","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":14432,"total_assignments":12657,"total_const_size":4530,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,4],"model_input_scales":[4],"module_sizes":{"kzg":[],"poseidon":[14432,[1]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}}],"required_range_checks":[[-8,8]],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1715153670916}


In [7]:
_, verifier_model = computation_to_model(computation, precal_witness_path, False, error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path, verifier_model, verifier_model_path)

In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 60.84906888008118 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 12.75
==== Generating Proof ====
proof:  {'instances': [['d1c9d1d20a90c893c336feef1eab835aefd88d5a68f68881423b9ce4635f520f', '0100000000000000000000000000000000000000000000000000000000000000', 'cc00000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x0deac0f635d86e17e45897c895861b2a8f10b9a77b30ce63bd8b30bf985fa48527f4b5a69a015e2e7d37fc2b2a793602b81890695858580a09ec6c8160dc85df21a042491d75282c89906ebc858e82eb9ad390bf053c725408deca2853b7d62f25e84296f591452cc9cd7f83b858c5f5b8a86efc911438af59e4d64fb4d3c13d132dc14fbc6d8c396255012e239aefbe96b5928a3e95b33ca306992270b8fefa186224affaaea3f40f3ef47f8422fd25c44c1e0e0e21d4fb4ece894f7d9b870319ec266be79d217b0fd713b42bf0bffd2a54f4d5aaea4131a9f47d43ac879af70a4e23d5763b25327a3ab9e471c3e7f77170a16ef0a9f56aef9141b8c80d48dd126bf82457943d943170ba45a82a2c4fb1347185c3de1d94d5c02320ea165bd926ed0a02697df29

In [9]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, data_commitment_path)
print("Verifier gets result:", res)

Verifier gets result: [12.75]
